<a href="https://colab.research.google.com/github/GabrielCostaDeOliveira/faceDetectionTraining/blob/main/faceDetectionDL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

from fastai.vision.all import *
import numpy as np

/home/thss/prog/aulas/ml/faceDetectionTraining/.venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
path = Path(os.path.join(os.getcwd(), '..', 'dataset'))
Path.BASE_PATH = path
path

Path('.')

In [ ]:
path.ls()

Verificando se as imagens não foram corrompidas

In [6]:
failed = verify_images(get_image_files(path))
failed

(#0) []

Eliminando as imagens que falharam

In [8]:
failed.map(Path.unlink)

(#0) []

todo

In [3]:
def img2bbox(path):
    return Path(str(path).rsplit('.', 1)[0] + '.txt')

In [4]:
def get_y(path: str):
    bbox = np.genfromtxt(img2bbox(path), dtype=np.int32)
    return tensor(bbox[:4])

In [5]:
db = DataBlock(
    blocks=(ImageBlock, BBoxBlock),
    get_items=get_image_files,
    get_y=get_y,
    splitter=FuncSplitter(lambda o: o.parent.parent.parent.name.endswith('val')),
    item_tfms=Resize(128),
)

In [6]:
dls = db.dataloaders(path)
dls.show_batch(max_n=6, figsize=(8, 6))

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.